In [1]:
import pandas as pd
import numpy as np

### 데이터를 이어 붙이는 제일 직관적인 방법은 위 아래로 연결하는 방법 -> concat

In [10]:
data1 = pd.DataFrame({ 'name' : ['a', 'b', 'c'],
                      'age' : [17, 20, 20], 
                      'gender' : [ 'male', 'male', 'female']})

data2 = pd.DataFrame({ 'name' : ['d', 'e', 'f'],
                      'age' : [17, np.nan , 20], 
                      'gender' : [ 'female', 'male', 'female']})

data3 = pd.DataFrame({ 'name' : ['a', 'b', 'c', 'd', 'e'],
                     'shcool' : ['middle', 'high', 'high', 'middle', 'university']})


data1과 data2를 위아래로 연결

In [16]:
df = pd.concat([data1, data2])
df

,name,age,gender
0,a,17.0,male
1,b,20.0,male
2,c,20.0,female
0,d,17.0,female
1,e,NaN,male
2,f,20.0,female


앞에 인덱스가 거슬린다. 이를 다시 재할당하면

In [18]:
df.reset_index()

,index,name,age,gender
0,0,a,17.0,male
1,1,b,20.0,male
2,2,c,20.0,female
3,0,d,17.0,female
4,1,e,NaN,male
5,2,f,20.0,female


앞에 인덱스가 생긴건 좋은데, (구)인덱스가 index라는 컬럼으로 붙어서 보관된다. 이게 원래 몇이었는지를 기억하기에는 좋을 지 몰라도, 지금은 불필요하니까 아예 버리는 옵션 (`drop=True`)으로 재할당을 하면

In [20]:
df = df.reset_index(drop=True)
df

,name,age,gender
0,a,17.0,male
1,b,20.0,male
2,c,20.0,female
3,d,17.0,female
4,e,NaN,male
5,f,20.0,female


이렇게 이쁘게 정돈이 된다. 여기에 data3 를 합쳐보자. data3 는 컬럼자체가 다르기 때문에 위아래로 붙이면 우리가 원하는 모양이 아니다. \\

물론 강제로 이어 붙이면 붙기는 붙는다. 컬럼은 df 의 컬럼과 data3 의 컬럼이 모두 포함되며 없는 정보는 NaN 으로 나타나게 된다.

In [21]:
pd.concat([df, data3])

,name,age,gender,shcool
0,a,17.0,male,NaN
1,b,20.0,male,NaN
2,c,20.0,female,NaN
3,d,17.0,female,NaN
4,e,NaN,male,NaN
5,f,20.0,female,NaN
0,a,NaN,NaN,middle
1,b,NaN,NaN,high
2,c,NaN,NaN,high
3,d,NaN,NaN,middle


우리가 원하는 건, 옆에 이어붙이는 건데, 이 경우 axis=1 을 사용해서, 단순 병합을 도전!

In [25]:
pd.concat([df, data3], axis=1)

,name,age,gender,name,shcool
0,a,17.0,male,a,middle
1,b,20.0,male,b,high
2,c,20.0,female,c,high
3,d,17.0,female,d,middle
4,e,NaN,male,e,university
5,f,20.0,female,NaN,NaN


애매모호한 결과가 나왔다. (뭔가 원하는데로 오른쪽에 붙기는 했는데, name column 이 2개가 됨), 사실 concat 의 axis=1 옵션은 조금 위험할 수 있는데, 이는 이름을 보고 맞춘게 아니라 그냥 index 순서에 따라서 오른쪽에 붙이기 때문이다. 지금같은 경우 운좋게 같은 name 이 같은 index를 가졌기 때문에 제대로 붙은 것 처럼 보이지만, 예를 들어 인덱스를 바꾼다음 붙이게 되면 다음과 같이 된다.


In [36]:
df.index = range(6, 0, -1)
pd.concat([df, data3], axis=1)


,name,age,gender,name,shcool
6,a,17.0,male,NaN,NaN
5,b,20.0,male,NaN,NaN
4,c,20.0,female,e,university
3,d,17.0,female,d,middle
2,e,NaN,male,c,high
1,f,20.0,female,b,high
0,NaN,NaN,NaN,a,middle



참고로 이렇게 같은 이름이 2개인 데이터 프레임에서 열 이름으로 추출하면 같은 열이 모두 나온다.

In [37]:
tmp = pd.concat([df, data3], axis=1)
tmp['name']

,name,name
6,a,NaN
5,b,NaN
4,c,e
3,d,d
2,e,c
1,f,b
0,NaN,a


따라서 이런 경우에는 단순 병합이 아니라 join 을 해야하는데, pandas 에서는 이를 merge 라는 메쏘드를 사용해서 제공한다.

이는 SQL 등에서 사용되는 join 과 상당히 비슷한데, how 에 따라서 여러가지 방법을 취할 수 있다. 

- inner : 양쪽 데이터 프레임에 모두 있는 데이터(행)만 취하는
- outer : 양쪽에 있는 모든 행을 취하는 
- left : 왼쪽으로 지정된 데이터 프레임에 있는 행만 
- right : 오른쪽에 지정된 데이터 프레임에 있는 행만

In [40]:
pd.merge( left=df, right=data3, how = 'inner', on = 'name')  # name을 기준으로 양쪽 데이터프레임에 모두 있는 데이터만 


,name,age,gender,shcool
0,a,17.0,male,middle
1,b,20.0,male,high
2,c,20.0,female,high
3,d,17.0,female,middle
4,e,NaN,male,university


In [45]:
df = pd.merge( left=df, right=data3, how='outer', on = 'name')
df


,name,age,gender,shcool
0,a,17.0,male,middle
1,b,20.0,male,high
2,c,20.0,female,high
3,d,17.0,female,middle
4,e,NaN,male,university
5,f,20.0,female,NaN


만약 두 컬럼에 공통된 기준은 있지만, 그 열의 이름이 서로 다르다면 (예를들어 왼쪽에서는 'Name', 오른쪽에서는 'name' 이라면) on 을 각기 left_on, right_on 으로 표현할 수 있다.

또한 여기서는 사용하지 않았지만, 고유 index가 없는 경우 합성 key를 만들 수 있는데 (여러개 column을 합쳐서) 이 경우 left_on = ['a', 'b', 'c'], right_on = ['A', 'B', 'C'] 이렇게 리스트로 넣어주면 된다.


### 누락값 처리 

위에서 NaN 으로 나오는 값들은, Not a Number 로, 사실상 NA 와 같은 누락값으로 생각해도 된다. 정확히는 NaN과 NA는 각기 Not a Number 와 Not available (또는 Not answered) 이지만 대부분 별 일 없으면 같이 취급하는게 보통이다. 이런 누락값을 처리하는 방법은 여러가지가 있지만 (평균으로 대치 등등) 제일 무난한 방법은 누락값이 있는 행을 날려버리는 것


먼저 누락값 인지 아닌지를 확인하기 위해 `pd.isna()` 를 사용한다.

In [46]:
pd.isna(df)

,name,age,gender,shcool
0,False,False,False,False
1,False,False,False,False
2,False,False,False,False
3,False,False,False,False
4,False,True,False,False
5,False,False,False,True


말 그대로 누락값인지를 물어보았기 때문에 True 는 NaN 이라는 뜻\
비슷한 이름으로 isnull 메쏘드도 제공하는데, 사실상 같은 결과가 나온다. 

In [48]:
pd.isnull(df)

,name,age,gender,shcool
0,False,False,False,False
1,False,False,False,False
2,False,False,False,False
3,False,False,False,False
4,False,True,False,False
5,False,False,False,True



누락값을 단순히 날려버리기 전에 누락값을 날리면 얼마나 많은 데이터가 날라갈 지 확인해보는게 중요하다. 
보통 행 단위로 날려버리기 때문에, 그 행에 누락값이 1개라도 있는지를 보는 방법이 많이 이용된다.

In [54]:
df.isna().sum(axis=0) # 각 column 별로 몇개의 NA가 있는지 확인
df.isna().sum(axis=1) # 각 행에 몇개의 NA가 있는지 확인 

0    0
1    0
2    0
3    0
4    1
5    1
dtype: int64

In [56]:
#이를 복합적으로 응용하여 전체 몇개의 NA가 있는지 도 확인 가능하다.
df.isna().sum(axis=0).sum()

2

또는 제대로 응답된 갯수를 `count()` 하여 역으로 확인하는 것도 괜찮다.

In [57]:
df.count()

name      6
age       5
gender    6
shcool    5
dtype: int64

누락값의 확인했다면, 이를 없애거나 채워넣어야 한다. 가장 쉬운 방법은 제거하는 방법이다.

In [58]:
df

,name,age,gender,shcool
0,a,17.0,male,middle
1,b,20.0,male,high
2,c,20.0,female,high
3,d,17.0,female,middle
4,e,NaN,male,university
5,f,20.0,female,NaN


위 데이터에서 누락값이 있는 행은 4, 5행이다. (index 기준)  이를 통으로 날려버리기 위해서는 `dropna` 메쏘드를 사용한다.

In [59]:
df.dropna()

,name,age,gender,shcool
0,a,17.0,male,middle
1,b,20.0,male,high
2,c,20.0,female,high
3,d,17.0,female,middle


만약 데이터가 너무 소중해서 날려버리는 결정이 어렵다면, 이를 다른 값으로 대치하는 방법도 있다.

- 값을 임의의 값으로 대치하기
- 전 값으로 대치하기
- 후 값으로 대치하기

먼저 na값을 대치하는 방법은 `fillna(x)` 를 사용하는 방법이다. 이는 모든 결측값에 x 를 대입한다.

In [60]:
df.fillna(0)

,name,age,gender,shcool
0,a,17.0,male,middle
1,b,20.0,male,high
2,c,20.0,female,high
3,d,17.0,female,middle
4,e,0.0,male,university
5,f,20.0,female,0


또는 이 전값을 사용하는 방법도 있다. 

In [61]:
df.ffill()

,name,age,gender,shcool
0,a,17.0,male,middle
1,b,20.0,male,high
2,c,20.0,female,high
3,d,17.0,female,middle
4,e,17.0,male,university
5,f,20.0,female,university


4행의 age 값이 3행의 age 값으로, 5행의 shcool 값이 4행의 school 값으로 대치되었다. 


In [ ]:
마찬가지로 후값으로 대치하는 방법도 있다.


In [62]:
df.bfill()

,name,age,gender,shcool
0,a,17.0,male,middle
1,b,20.0,male,high
2,c,20.0,female,high
3,d,17.0,female,middle
4,e,20.0,male,university
5,f,20.0,female,NaN


이 경우 4행의 age 값이 5행의 age 값으로 대치되었지만, 5행의 shcool 값은 대치되지 못했다. (6행이 없어서..)

누락값이 있는 상태에서 무언가를 계산할 때에는 주의해야한다.

In [63]:
df

,name,age,gender,shcool
0,a,17.0,male,middle
1,b,20.0,male,high
2,c,20.0,female,high
3,d,17.0,female,middle
4,e,NaN,male,university
5,f,20.0,female,NaN


In [67]:
df['age'].mean() 

18.8

위 값은 NaN 을 자연스럽게 skip 한 결과이다. (통상 연산시 1개라도 NaN이 들어가면 전체가 NaN 이 된다.)